In [101]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from google.colab import drive
import nltk
import re
from string import punctuation
import numpy as np

In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
df= pd.read_excel('/content/drive/MyDrive/DM_lab/Lab test/Text_Clustering.xlsx')


In [34]:
df.dropna(subset=['text'], how='all', inplace=True)

In [7]:
df.head()

,text,subject
0,The idea is to create a ticketing support syst...,NaN
1,Level 1 - Integrate Machine Learning Algorit...,NaN
2,\nCustomers could be well informed in advance ...,NaN
3,Have a high level categorization of the histor...,NaN
4,Get information from customer on the systems...,NaN


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260 entries, 0 to 260
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   text     260 non-null    object 
 1   subject  0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 6.1+ KB


In [15]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
pattern = re.compile(r'\b(' + r'|'.join(stopwords) + r')\b\s*')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [36]:
def text_clean(text):
  try:
    text = pattern.sub('', text.lower())
    cleaned_text = re.sub(r'[^a-z ]+', ' ', text)
    return cleaned_text 
  except:
    print(text)
    return ''

In [37]:
df['text']= df['text'].apply(text_clean)

In [40]:
df['tokens']=df['text'].apply(lambda x:x.split())

In [48]:
stemm = PorterStemmer()

def stem_conv(tokens):
    stems =[]
    for token in tokens:
        stems.append(stemm.stem(token))
    return stems
df['stem'] = df['tokens'].apply(stem_conv)

In [49]:
df['stem_text'] = df['stem'].apply(lambda x:" ".join(x))

In [80]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['stem_text'])

In [85]:

model = KMeans(n_clusters=7, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(max_iter=100, n_clusters=7, n_init=1)

In [129]:
print("Cluster centroids")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(7):
    print("Centroid for Cluster {}: {}".format(i,terms[order_centroids[i][0]]))
    print(terms[order_centroids[i][0]])


Cluster centroids
Centroid for Cluster 0: issu
issu
Centroid for Cluster 1: user
user
Centroid for Cluster 2: credit
credit
Centroid for Cluster 3: ticket
ticket
Centroid for Cluster 4: data
data
Centroid for Cluster 5: mar
mar
Centroid for Cluster 6: db
db


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [143]:
new_text = "You should have told me yesterday" 
new_text_clean=text_clean(new_text)
new_stem=stem_conv(new_text_clean.split())
new_stem=" ".join(stem_conv(new_text_clean.split()))
Y= vectorizer.transform([new_stem])
prediction = model.predict(Y)
print(prediction)

[1]
